# QF620 Project | Part 3  (Static Replication)

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm
from scipy import integrate
from math import log, sqrt, exp

In [5]:
##payoff function:
##Payoff=ST**3*10**(-8)+0.5*np.log(ST)+10

## I.Preparation

### 1.1Evaluation period(T), discount rate(r) and start value(S0)

In [6]:
## T
import datetime as dt
today = dt.date(2013, 8, 30)
expiry = dt.date(2015, 1, 17)
T = (expiry-today).days/365.0
T

1.3835616438356164

In [7]:
## r
discount=pd.read_csv("discount.csv")
discount.head()

,Day,Rate (%)
0,7,0.14981
1,14,0.17250
2,19,0.17595
3,47,0.23118
4,82,0.25150


In [8]:
#Lianear introplonation
r1=0.38802
r2=0.43892
r=((505-474)/(565-474)*r1+(565-505)/(565-474)*r2)/100
r

0.004215804395604396

In [9]:
## S0:Google stock price on 30-Aug-2013
S0=846.9

### 1.2Parameter(sigma)

In [10]:
##we use ATM implied volatility derived from Part2 as sigma in valuing the contract
ATMvol_BS=0.258

## II.Use different model to determine the price of this contract

### 1. Black-Scholes Model

\begin{equation}
V0 = e^{-rT} [{{S_0^3} \times\ {\rm e}^{3(r-\frac{\sigma^2}{2})T} \times\ 10^{-8} \times\ {\rm e}^{\frac{9}{2}{\sigma^2}T} + \frac{1}{2}log(S_0) + \frac{1}{2}{(r-\frac{\sigma^2}{2})T} + 10}]
\end{equation}

In [11]:
##ST=S0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*x)
def contract_BS(S0,T,r,sigma):
    V0=np.exp(-r*T)*(S0**3*np.exp(3*(r-0.5*sigma**2)*T)*10**(-8)*np.exp(4.5*T*sigma**2)+0.5*np.log(S0)+0.5*(r-0.5*sigma**2)*T+10)
    return V0

In [12]:
contract_BS(S0,T,r,ATMvol_BS)

21.374293191359243

### 2. Bachelier Model

\begin{equation}
\begin{split}
V0 &= e^{-rT} [\frac{1}{2}log(S_0) + 10] + e^{-rT} \times\ {S_0^3} \times\ 10^{-8} \times\ f_1(x) + e^{-rT} \times\ {\frac{1}{2}} \times\ f_2(x) \\
         f_1(x) &= \int_{-\frac{1}{\sigma\sqrt{T}}}^t ({1 + \sigma\sqrt{T}x}^3) \times\ \frac{1}{\sqrt{2\pi}} 
         \times\ e^{\frac{x^2}{2}}\,{\rm d}x, 
         \hspace{2cm} f_2(x) = \int_{-\frac{1}{\sigma\sqrt{T}}}^t log({1 + \sigma\sqrt{T}x}) \times\ \frac{1}{\sqrt{2\pi}} \times\ e^{\frac{x^2}{2}}\,{\rm d}x\\
\end{split}
\end{equation}

In [13]:
a=float("inf")
b=float("-inf")

In [102]:
##The Probability of payoff: P{ST>0}=P{S0+S0*sigma*sqrt(T)*x>0} = P{x>(-1/(sigma*sqrt(T)))}

In [14]:
sigma1=ATMvol_BS
sigma1

0.258

In [15]:
## solution of f1(x)
def f1(x):
    return (1/np.sqrt(2*np.pi))*(1+sigma1*np.sqrt(T)*x)**3*np.exp(-0.5*x**2)
v1,err=integrate.quad(f1,-1/(sigma1*np.sqrt(T)),a)
print(v1)    

1.2762874808900895


In [16]:
##solution of f2(x)
def f2(x):
    return (1/np.sqrt(2*np.pi))*np.log(1+sigma1*np.sqrt(T)*x)*np.exp(-0.5*x**2)
v2,err=integrate.quad(f2,-1/(sigma1*np.sqrt(T)),a)
print(v2)  

-0.054701916214791924


In [17]:
##valuation of contract 
def contract_Bachelier(S0,r):
    V0=np.exp(-r*T)*(0.5*np.log(S0)+10)+np.exp(-r*T)*S0**3*(10**(-8))*v1+np.exp(-r*T)*0.5*v2
    return V0
    print(V0)    

In [18]:
contract_Bachelier(S0,r)

20.97330518544933

### 3.Valuation of the contract(print result)

In [19]:
print("Based on Black-Scholes Model, \nThe value of this contract:{:.4f} \nsigma:{:.4f}".format(contract_BS(S0,T,r,ATMvol_BS),ATMvol_BS))
print("\nBased on Bachelier Model, \nThe value of this contract:{:.4f} \nsigma:{:.4f}".format(contract_Bachelier(S0,r),sigma1))

Based on Black-Scholes Model, 
The value of this contract:21.3743 
sigma:0.2580

Based on Bachelier Model, 
The value of this contract:20.9733 
sigma:0.2580
